# ПРОЕКТ для "Викишоп"

Я работаю в интернет-магазине «Викишоп» и мы запускаем новый сервис: \
Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. \
Назрела необходимость в инструменте, который будет искать токсичные комментарии и отправлять их на модерацию. 

**Цель проекта**: построить модель, которая умеет классифицировать комментарии на позитивные и негативные.
- Метрика качества F1 должна быть не меньше 0,75. 

**Ход исследования:**

*Шаг 1* - Загрузка и подготовка данных;\
*Шаг 2* - Обучение нескольких моделей;\
*Шаг 3* - Вывод.

## Шаг 1. Загрузка и подготовка данных

In [ ]:
!pip install torch -q
!pip install wordcloud -q
!pip install transformers -q
!pip install hf_xet -q

In [ ]:
# импоритруем pandas для обработки, анализа и структурирования данных
import pandas as pd 
# импоритруем numpy для работы с данными
import numpy as np
# так же импоритруем matplotlib.pyplot для будущего построения графиков
import matplotlib.pyplot as plt

import nltk
import re
import spacy

from tqdm import tqdm, notebook
tqdm.pandas()
from nltk.corpus import stopwords as nltk_stopwords
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.metrics import f1_score
from sklearn.model_selection import (
    GridSearchCV, 
    train_test_split,
    cross_val_score
)

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from wordcloud import WordCloud
from catboost import CatBoostClassifier 

import torch
import transformers 
from transformers import AutoTokenizer, AutoModel

import warnings
warnings.filterwarnings("ignore")


pd.set_option('display.max_columns', 20)
pd.set_option('display.max_rows', 20)

In [ ]:
# зафиксируем константы:
RANDOM_STATE = 42

In [ ]:
!python -m spacy download en_core_web_sm
nltk.download('stopwords')

### 1.1 Загрузим данные из csv-файла в датафрейм c помощью библиотеки pandas

In [ ]:
try: # открываем наш файл с данными в среде JupiterHUB:
    data_toxic = pd.read_csv('/datasets/toxic_comments.csv', index_col = 0) 
        
except: # либо берем данные на ПК для локальной версии Jupiter:
    data_toxic = pd.read_csv('C://Users//Voova//datasets//toxic_comments.csv', index_col = 0) 

Описание данных:\
**Признак**
- **text** — содержит текст комментария;

**Целевой признак**
- **toxic** — целевой признак.

### 1.2 Изучим общую информацию о датафрейме. Выведим первые строки набора данных.

In [ ]:
data_toxic.info()

Перед нами датафрейм на **2** колонки и **159 292** строки. Пропущенные значения отсутствуют. Названия столбцов соответствуют общепринятым нормам. \
Посмотрим первые 5 строчек таблицы:

In [ ]:
data_toxic.head(5)

Данные в таблице соответствуют описанию, типы данных корректные.  

Проверим соотношение токсичных отзывов к нетоксичным:

In [ ]:
balance = 100 * data_toxic['toxic'].value_counts()[1] /(data_toxic['toxic'].value_counts()[0] + data_toxic['toxic'].value_counts()[1])
print(f'Токсичные отзывы составляют: {balance.round(2)} % от всего датасета')

Данные несбалансированы, учтем это в параметре стратификации при разбивке на выборки. 

## 1.3 Проверим датасет на дубликаты:

In [ ]:
data_toxic.duplicated().sum() # выведим сумму явных дубликатов:

- Дубликаты отсутствуют. 

## 1.4 Предобработка текста перед обучением: 

Перед лемматизированием, нужно оставить в отзыве только латинские символы и пробелы. Чтобы их найти, воспользуемся встроенным модулем `re` (сокр. от regular expressions) в функции очистки текста:

In [ ]:
def clear_text(text):
    text = text.lower()
    return " ".join((re.sub(r'[^a-zA-Z]', ' ', text)).split())

Добавим в датасет столбец с очищенным текстом: 

In [ ]:
data_toxic['clear_text'] = data_toxic['text'].progress_apply(clear_text)

In [ ]:
data_toxic.head(10)

Напишем функцию и произведем лемматизацию текста (приведем слова к их базовой форме).\
Для лемматизации я выбираю библиотеку `spaCy`, так как она уже учитывает предварительную токенизациею, и POS-тегирование. 

In [ ]:
nlp = spacy.load('en_core_web_sm')
def lemmatize(text):
    doc = nlp(text)
    return " ".join([token.lemma_ for token in doc])

Добавим в датасет столбец с лемматизированным текстом:

In [ ]:
data_toxic['lemmatize_text'] = data_toxic['clear_text'].progress_apply(lemmatize)

In [ ]:
data_toxic.head()

### 1.5 Посмотрим на частоту встречающихся токсичных слов:  

In [ ]:
bad_text = " ".join(data_toxic[data_toxic['toxic'] == 1]['lemmatize_text'].tolist())

In [ ]:
bad_word = WordCloud(width = 800, 
                     height = 400, 
                     background_color = 'black', 
                     collocations = False
                    ).generate(bad_text)

In [ ]:
plt.figure(figsize=(10, 5))
plt.imshow(bad_word, interpolation = 'bilinear')
plt.title('Наиболее часто встречающиеся токсичные слова', fontsize = 16)
plt.axis('off')
plt.show()

Самое популярное токсичное слово обнаружено. Вот, что интересно, а если в описании товара написано, что он `OHUITEL'NIY` (если бы мы были в комментариях RU сегмента - получается комментарий посчитается токсичным ? 

### Вывод: 
На этапе загрузки и поготовки данных, мы обнаружили, что в нашем распоряжении датасет на **2** столбца и **159 292** строки, пропуски и дубликаты в данных отсутствуют. \
Текст очищен, лемматизирован и готов к моделированию.  

## Шаг 2. Обучение нескольких моделей

### 2.1 Разделение данных на обучающую и тестовую выборки

In [ ]:
X = data_toxic['lemmatize_text']
y = data_toxic['toxic']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
                                                    X,
                                                    y, 
                                                    test_size = 0.25, 
                                                    random_state = RANDOM_STATE,
                                                    stratify = y # так как данные у нас несбалансированы
                                                    ) 

Размеры выборок:

In [ ]:
print(X_train.shape[0])
print(y_train.shape[0])
print(X_test.shape[0])
print(y_test.shape[0])

Преобразуем корпус текстов в мешок слов (обозначим стоп слова и составим матрицу): 

In [ ]:
stopwords = list(set(nltk_stopwords.words('english')))

In [ ]:
count_tf_idf = TfidfVectorizer(stop_words = stopwords) 

In [ ]:
tf_idf_train = count_tf_idf.fit_transform(X_train)
tf_idf_test = count_tf_idf.transform(X_test)

Данные подготовлены, приступим к обучению: 

### 2.2 LogisticRegression()

In [ ]:
model_lr = LogisticRegression()
params_grid_lr = { 
    'C': [1, 10],
    'max_iter': [150],
    'random_state' : [RANDOM_STATE]
}

In [ ]:
%%time
grid_cv = GridSearchCV(
                       model_lr, 
                       params_grid_lr, 
                       scoring ='f1', 
                       cv = 3)
grid_cv.fit(tf_idf_train, y_train)
print('Лучшие параметры: ', grid_cv.best_params_)
print('F1 значение на тренеровочной выборке: {:.2f}'.format((grid_cv.best_score_)))

Метрика F1 как раз нам подходит, так как данные несбалансированы и нам нужно поймать баланс между *precision* и *recall*.

### 2.3 CatBoostClassifier()

In [ ]:
model_cat = CatBoostClassifier()
params_grid_cat = {
    'n_estimators' : [20, 50, 80],
    'random_state' : [RANDOM_STATE],
    'max_depth' : [3, 4, 5]
}

In [ ]:
%%time

grid_cv_cat = GridSearchCV(
                           estimator = model_cat, 
                           param_grid = params_grid_cat, 
                           scoring ='f1', 
                           cv = 3
                          )
grid_cv_cat.fit(tf_idf_train, y_train)
print('Лучшие параметры: ', grid_cv_cat.best_params_)
print('F1 значение на тренеровочной выборке: {:.2f}'.format((grid_cv_cat.best_score_)))

Проверим лучшую модель на тестовой выборке:

In [ ]:
%%time
 
model_lr = LogisticRegression(
                              C = 10, 
                              max_iter = 150, 
                              random_state = RANDOM_STATE
)

model_lr.fit(tf_idf_train, y_train)
predictions_lr = model_lr.predict(tf_idf_test)
print('F1 значение на тестовой выборке: {:.2f}'.format(f1_score(predictions_lr, y_test)))

Метрика F1 удовлетворяет условию задачи, на тренировочных данных обе модели справляются почти одинаково хорошо - регрессия немного точнее. 

### Вывод без BERT:
В данном исследовании перед нами стояла задача разработать решение, которое позволит классифицировать комментарии на позитивные и негативные.
- Нам были предоставлены данные в датафрейме **data_toxic** c **2** столбцами и **159 292** строками. \
Данные предоставлены без пропусков и дубликатов.
- Далее мы оставили в тексте только латинские символы и пробелы, воспользовавшись встроенным модулем **re** (сокр. от regular expressions) в функции очистки текста.
- После этого произвели лемматизацию текста.
- На следующем этапе мы выяснили, что наиболее часто встречающееся токсичное слово - **FUCK**.
- После разделения данных на обучающую и тестовые выборки, нами были обучены 2 модели:

1. Модель **LogisticRegression()** на тестовых данных показала значение метрики F1 равное **0.77**.
2. Модель **CatBoostClassifier()** на тестовых данных показала значение метрики F1 равное **0.73**.

- На тренировочных данных протестировали лучшую модель, где *LogisticRegression()* показала значение метрики F1 равное **0.78**, что удовлетворяет условию задачи.

- Далее пытался попробовать в BERT, но всё мимо :(

### 2.3 BERT

In [ ]:
try: # открываем наш файл с данными в среде JupiterHUB:
    data_bert = pd.read_csv('/datasets/toxic_comments.csv', index_col = 0) 
        
except: # либо берем данные на ПК для локальной версии Jupiter:
    data_bert = pd.read_csv('C://Users//Voova//datasets//toxic_comments.csv', index_col = 0) 

In [ ]:
# инициализируем токенизатор
#model_name = "unitary/toxic-bert"
model_name = "bert-base-uncased"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)


#tokenized = data_bert['text'].apply(
#    lambda x: tokenizer.encode(x, add_special_tokens = True))

#max_len = 0
#for i in tokenized.values:
#    if len(i) > max_len:
#        max_len = len(i)

#padded = np.array([i + [0]*(max_len - len(i)) for i in tokenized.values])

#attention_mask = np.where(padded != 0, 1, 0)

data_bert = data_bert.sample(2000, random_state = 42)
data_bert['tokenized_text'] = data_bert['text'].apply(
    lambda x: tokenizer.encode(x, add_special_tokens=True, truncation=True, max_length=512)
)
max_len = 512
data_bert['padded_text'] = data_bert['tokenized_text'].apply(
    lambda x: x + [0] * (max_len - len(x))
)
data_bert['attention_mask'] = data_bert['padded_text'].apply(
    lambda x: [1 if token != 0 else 0 for token in x]
)
padded = np.array(data_bert['padded_text'].tolist())
attention_mask = np.array(data_bert['attention_mask'].tolist())

batch_size = 100
embeddings = []

for i in notebook.tqdm(range(padded.shape[0] // batch_size + 1)):
    batch_padded = torch.LongTensor(padded[batch_size*i:batch_size*(i+1)])
    attention_mask_batch = torch.LongTensor(attention_mask[batch_size*i:batch_size*(i+1)])
    
    with torch.no_grad():
        batch_embeddings = model(batch_padded, attention_mask=attention_mask_batch)
    
    embeddings.append(batch_embeddings[0][:, 0, :].numpy())

In [ ]:
%%time

model.eval()

# токенизация
encodings = tokenizer(
    data_bert['text'].tolist(),
    padding = True,
    truncation = True,
    max_length = 512,
    return_tensors = 'pt'
)

batch_size = 50
embeddings = []

# обработка батчами
for i in notebook.tqdm(range(0, len(data_bert), batch_size)):
    batch_input_ids = encodings['input_ids'][i:i+batch_size]
    batch_attention_mask = encodings['attention_mask'][i:i+batch_size]
    
    with torch.no_grad():
        outputs = model(
            input_ids = batch_input_ids,
            attention_mask = batch_attention_mask
        )
        # получение [CLS] эмбеддингов
        cls_embeddings = outputs.last_hidden_state[:, 0, :]
        embeddings.append(cls_embeddings.numpy())

# соберём все эмбеддинги в матрицу признаков вызовом функции concatenate()
features = np.concatenate(embeddings, axis=0)

In [ ]:
X = features
y = data_bert['toxic']
 

X_train, X_test, y_train, y_test = train_test_split(
                                                    X, 
                                                    y, 
                                                    test_size = 0.5, 
                                                    random_state = 42
)

In [ ]:
%%time
 
model = LogisticRegression(random_state = 42)
# обучим модель
model.fit(X_train, y_train)
predictions = model.predict(X_test)
print('F1 значение на тестовой выборке: {:.2f}'.format(f1_score(predictions, y_test)))